In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-core langchain_openai python-dotenv langsmith pydantic spotipy

In [2]:
%pip install --quiet -U jupyterlab-lsp
%pip install --quiet -U "python-lsp-server[all]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
## Setup logging
import logging
import os
from dotenv import load_dotenv

load_dotenv(override=True)
logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the format
    handlers=[logging.StreamHandler()]  # Output to the console
)

# Define Search Tool

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=50,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
)
name = search_tool.get_name()
desc = search_tool.description
desc

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

# Create ToolNode

In [5]:
from langgraph.prebuilt import ToolNode
from spotify_tools import get_playlists, create_spotify_playlist, add_tracks_to_playlist, filter_artists, get_artists_from_playlist, find_similar_artists, find_top_tracks
from plan import validate_plan

tools = [get_playlists, create_spotify_playlist, add_tracks_to_playlist, filter_artists, validate_plan, get_artists_from_playlist, find_similar_artists, find_top_tracks, search_tool]
tool_node = ToolNode(tools)

# Bind Tools to Model

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"), temperature=1.0)
# llm_with_tools = llm.bind_tools(tools, strict=True, parallel_tool_calls=False)
llm_with_tools = llm.bind_tools(tools, strict=True)

# System Prompt

In [7]:
from langchain_core.messages import SystemMessage
from prompts import Prompts

system_message = SystemMessage(Prompts.SYSTEM)


# First Message

In [8]:
from langchain_core.messages import HumanMessage
from prompts import Prompts

human_message = HumanMessage(Prompts.SPOTIFY)
llm_response = llm_with_tools.invoke([system_message, human_message])

2024-11-15 12:37:24,575 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [9]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "validate_plan":
    raise ValueError("should be tool message calling 'validate_plan'")

In [10]:
llm_response.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  validate_plan (call_fjz5SftioOKJY9wSZNp7vzul)
 Call ID: call_fjz5SftioOKJY9wSZNp7vzul
  Args:
    plan: {'steps': [{'step_number': 1, 'name': 'initiate-playlist-creation', 'type': 'action', 'description': "Retrieve playlist details for 'New Rock and Blues' to identify the playlist ID and existing artists.", 'success_criteria': 'Successfully retrieved playlist ID and list of artists.', 'tool': 'functions.get_playlists', 'action': "Get the Spotify playlists and find 'New Rock and Blues'. Extract playlist ID and artists.", 'condition': None, 'substeps': None}, {'step_number': 2, 'name': 'retrieve-existing-artists', 'type': 'action', 'description': "Get the list of artists from 'New Rock and Blues' playlist using the playlist ID.", 'success_criteria': 'List of existing artists successfully retrieved.', 'tool': 'functions.get_artists_from_playlist', 'action': "Retrieve existing artists from 'New R

In [11]:
plan = llm_response.tool_calls[0]["args"]
plan

{'plan': {'steps': [{'step_number': 1,
    'name': 'initiate-playlist-creation',
    'type': 'action',
    'description': "Retrieve playlist details for 'New Rock and Blues' to identify the playlist ID and existing artists.",
    'success_criteria': 'Successfully retrieved playlist ID and list of artists.',
    'tool': 'functions.get_playlists',
    'action': "Get the Spotify playlists and find 'New Rock and Blues'. Extract playlist ID and artists.",
    'condition': None,
    'substeps': None},
   {'step_number': 2,
    'name': 'retrieve-existing-artists',
    'type': 'action',
    'description': "Get the list of artists from 'New Rock and Blues' playlist using the playlist ID.",
    'success_criteria': 'List of existing artists successfully retrieved.',
    'tool': 'functions.get_artists_from_playlist',
    'action': "Retrieve existing artists from 'New Rock and Blues' playlist using its ID.",
    'condition': None,
    'substeps': None},
   {'step_number': 3,
    'name': 'find-simil

# chat_prompt_template holds all messages (Human, AI, Tool)

In [12]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt_template: ChatPromptTemplate = system_message + human_message + llm_response
messages = chat_prompt_template.format_messages()
messages

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

# Check for Tools in Message

In [13]:
from langchain_core.messages import ToolMessage

tools_in_ai_message = []
tools_by_name = {tool.name: tool for tool in tools}
messages = chat_prompt_template.format_messages()
for tool_call in messages[-1].tool_calls:
    tool = tools_by_name[tool_call["name"]]
    tools_in_ai_message.append(tool)
    # observation = tool.invoke(tool_call["args"])
    # result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
if len(tools_in_ai_message) == 0:
    raise ValueError
else:
    print(tools_in_ai_message)

[StructuredTool(name='validate_plan', description='Validates a step-by-step plan to solve a problem\n\nArgs:\n    plan (Plan): a step-by-step plan to solve a problem\n\nReturns:\n    bool: whether plan is okay or not', args_schema=<class 'langchain_core.utils.pydantic.validate_plan'>, func=<function validate_plan at 0x0000027B42783EC0>)]


# Tool Call (should be validate_plan())

In [14]:
tool_response = tool_node.invoke({"messages": messages})
tool_message = tool_response["messages"][0]
tool_message.pretty_print()

================================= Tool Message =================================
Name: validate_plan

true


# Add Tool Response to Messages

In [15]:
chat_prompt_template += tool_message
chat_prompt_template.format_messages()

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

# Send Messages with Tool result to LLM

In [16]:
llm_response = llm_with_tools.invoke(chat_prompt_template.format_messages())
llm_response.pretty_print()

2024-11-15 12:37:25,946 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  get_playlists (call_3tCY4MxOXny24cxOhvU3t9oB)
 Call ID: call_3tCY4MxOXny24cxOhvU3t9oB
  Args:


# Validate correct tool call

In [17]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "get_playlists":
    raise ValueError("should be tool message calling 'validate_plan'")

# Tool Call (should be get_playlists())

In [18]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
tool_response = tool_node.invoke({"messages": messages})

In [19]:
from langchain_core.messages import ToolMessage

tool_message: ToolMessage = tool_response["messages"][0]
tool_message.pretty_print()

================================= Tool Message =================================
Name: get_playlists

[{"id": "2dbYK5b7J0F7IdH5n1TEUK", "name": "RPreacher"}, {"id": "4ack9YtUhdxRayJDAqlfQe", "name": "RP Bossa Nova Chill "}, {"id": "75NW18NgdZuZeifrcjxKlZ", "name": "GVF"}, {"id": "5iyONtUO21O88xw8pBblwh", "name": "Now And Then"}, {"id": "4SDSUMg2HJcGFmHlupsU7U", "name": "Jazz classics"}, {"id": "5fBxLG2wLAPVpx83402rHl", "name": "Acdc"}, {"id": "17Yxm5hQbkbZwDcCTuiDXC", "name": "einaudi"}, {"id": "2X6GccWfIqPWO6VYx405nO", "name": "Eric Clapton Blues"}, {"id": "3smVXe8y8w8nt4lFH0HBkW", "name": "Alchemy: Dire Straits"}, {"id": "2ifRZBXnX7UmIPUlGet5WZ", "name": "Blues"}, {"id": "5nn2ePtd9gYEEDL3XnACCB", "name": "brazil"}, {"id": "6YAntoKNXuogPKdV26b6XE", "name": "CCR"}, {"id": "2VHkZvOMCkLF3B7TsKerKD", "name": "Chill"}, {"id": "508lbLx9XUkHv5Xb45hxTP", "name": "Classic Rock"}, {"id": "7zzDMq1sPwkZPYeqOQJ9qS", "name": "The New Four Seasons - Vivaldi Recomposed"}, {"id": "4l8WcaYlPkuVPXVGxMIc

# Add Tool Response to Messages

In [20]:
chat_prompt_template += tool_message
messages = chat_prompt_template.format_messages()

# Send Playlists Results to LLM

In [21]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

2024-11-15 12:37:28,200 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  get_artists_from_playlist (call_NfxrmJuZpVakzy9R1zWg2GoP)
 Call ID: call_NfxrmJuZpVakzy9R1zWg2GoP
  Args:
    playlist_id: 01tk0aitEuGK0ajWCkzdKc


# Validate correct tool call

In [22]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "get_artists_from_playlist":
    raise ValueError("should be tool message calling 'get_artists_from_playlist'")

# Add AI Tool Call to Messages

In [23]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()

# Tool Call (get_artists_from_playlist())

In [24]:
tool_response = tool_node.invoke({"messages": messages})
tool_message: ToolMessage = tool_response["messages"][0]
tool_message.pretty_print()

================================= Tool Message =================================
Name: get_artists_from_playlist

{"01aC2ikO4Xgb2LUpf9JfKp": "Gary Clark Jr.", "1iEaqWaYpKo9x0OrEq7Q7z": "Ayron Jones", "0RqtSIYZmd4fiBKVFqyIqD": "Thirty Seconds To Mars", "7jdFEYD2LTYjfwxOdlVjmc": "KALEO", "7Ln80lUS6He07XvHI8qqHH": "Arctic Monkeys", "4pejUc4iciQfgdX6OKulQn": "Queens of the Stone Age", "0Nrwy16xCPXG8AwkMbcVvo": "Black Pistol Fire", "1xKrH6GSh9CJh8nYwbqW7B": "Welshly Arms", "6vYg01ZFt1nREsUDMDPUYX": "The Record Company", "0bZCak2tcRMY1dzEIuwF42": "The Heavy", "4NpFxQe2UvRCAjto3JqlSl": "Greta Van Fleet", "26opZSJcXshCmCwxgZQmBc": "Whiskey Myers", "3xYXYzm9H3RzyQgBrYwIcx": "Colter Wall", "13ZEDW6vyBF12HYcZRr4EV": "Tyler Childers", "4YLtscXsxbVgi031ovDDdh": "Chris Stapleton", "5VPCIIfZPK8KPsgz4jmOEC": "The Blue Stones", "1l9I7G8J8AnMScWQwlNJ4M": "Goodbye June", "12Chz98pHFMPJEknJQMWvI": "Muse", "5HsS48kuvghKcNpwOaAvB5": "Samantha Fish", "7mnBLXK823vNxN3UWB7Gfz": "The Black Keys", "3mVWMgLc7bcyC

In [25]:
# json.loads(track_list_tool_message.content)

# Add Tool Response to Messages

In [26]:
chat_prompt_template += tool_message
messages = chat_prompt_template.format_messages()
messages

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

# Send Tool Result to LLM

In [27]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

2024-11-15 12:37:38,115 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  find_similar_artists (call_aoxfU9e0DFhqoyWkcwPt83Ux)
 Call ID: call_aoxfU9e0DFhqoyWkcwPt83Ux
  Args:
    artists: ['01aC2ikO4Xgb2LUpf9JfKp']
  find_similar_artists (call_av1b7yvuV7Yhv2txEj1YBm66)
 Call ID: call_av1b7yvuV7Yhv2txEj1YBm66
  Args:
    artists: ['1iEaqWaYpKo9x0OrEq7Q7z']
  find_similar_artists (call_3UYu6vFdxexiHuXmKbKsDNeB)
 Call ID: call_3UYu6vFdxexiHuXmKbKsDNeB
  Args:
    artists: ['0RqtSIYZmd4fiBKVFqyIqD']
  find_similar_artists (call_TKFEowMVerSDNgUaG6SvM72r)
 Call ID: call_TKFEowMVerSDNgUaG6SvM72r
  Args:
    artists: ['7jdFEYD2LTYjfwxOdlVjmc']
  find_similar_artists (call_ZShJLHNscohFUEJroE3pGhNo)
 Call ID: call_ZShJLHNscohFUEJroE3pGhNo
  Args:
    artists: ['7Ln80lUS6He07XvHI8qqHH']
  find_similar_artists (call_TSP8KbiDxKbgD5yh52ajirYI)
 Call ID: call_TSP8KbiDxKbgD5yh52ajirYI
  Args:
    artists: ['4pejUc4iciQfgdX6OKulQn']
  find_similar_artists (call_aUWpcJT64IKY9QFfqKmQ

# AI Tool Call (find_similar_artist())

In [28]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "find_similar_artists":
    raise ValueError("should be tool message calling 'find_similar_artists'")

# Add AI Tool Call to Messages

In [29]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
messages

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

# Tool Call (find_similar_artists())

In [30]:
tool_response = tool_node.invoke({"messages": messages})
for tool_message in  tool_response["messages"]:
    tool_message.pretty_print()

================================= Tool Message =================================
Name: find_similar_artists

{'72Pauuctba5lMzC89R0Vk0', '5jMGnqJkgPaiJzwy5bOcYX', '6vYg01ZFt1nREsUDMDPUYX', '0tgaHqkU1p7QhBUIzKXVU9', '0Nrwy16xCPXG8AwkMbcVvo', '7mnBLXK823vNxN3UWB7Gfz', '4fXkvh05wFhuH77MfD4m9o', '1riHqX633Kup3mJAw8WR8p', '4xac3zhHlBm5QDxbZeqgeR', '5kuJibJcwOC53s3OkoGMRA', '02seUFsFQP7TH4hLrTj77o', '16GcWuvvybAoaHr0NqT8Eh', '5fsDcuclIe8ZiBD5P787K1', '356c8AN5YWKvz86B4Sb1yf', '5krkohEVJYw0qoB5VWwxaC', '1Jl8u1U1GtBlwocZK5LWZv', '6YWdHD3R863Apw1hkx3BwC', '2gFsmDBM0hkoZPmrO5EdyO', '40N10exWtaCVUtBMftQn3t', '521Jw1ixgckc0WJHhm2Klu'}
================================= Tool Message =================================
Name: find_similar_artists

{'0nhDd1RWjZ6SDV1Vg1Ku2Q', '3SMJIplWXJ4qwuuGRHZyJH', '3pigXV2QUatr3szcb3MacR', '7LO9DGtrr5LrhQEbhRontJ', '4ygqEKxbe61knnDaodGFmi', '6HySSUk2KthFj3bd9R61ql', '3aVd0CcXBa0rOlxMpUS6on', '5NokRbYYfqacBmRVBRj0wD', '48vfT7i3EmuNEDAVRhzxpp', '0slSgjqTuj6hcod6IcG6zu', 

# Add Tool Response to Messages

In [31]:
chat_prompt_template += tool_response["messages"]
messages = chat_prompt_template.format_messages()
messages

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

# Send Tool Result to LLM

In [32]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

2024-11-15 12:37:54,343 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  filter_artists (call_U7VFDTRkS153WwO9M4HRxx9x)
 Call ID: call_U7VFDTRkS153WwO9M4HRxx9x
  Args:
    playlist_id: 01tk0aitEuGK0ajWCkzdKc
    new_artists: ['7mnBLXK823vNxN3UWB7Gfz', '356c8AN5YWKvz86B4Sb1yf', '0Nrwy16xCPXG8AwkMbcVvo', '5tFRohaO5yEsuJxmMnlCO9', '5kuJibJcwOC53s3OkoGMRA', '1l9I7G8J8AnMScWQwlNJ4M', '0XBRd3N11rCbh0s8qHf41m', '6VDdCwrBM4qQaGxoAyxyJC', '16GcWuvvybAoaHr0NqT8Eh', '7MhMgCo0Bl0Kukl93PZbYS', '0yb46jwm7gqbZXVXZQ8Z1e', '1JnWgHswLNLBcTnAOV8lA9', '1NlHgCXes12iEkqkXdvk4n', '7LIy7KinYq7a83dUH6kvxT', '4zYQWYmtimAEmI6WWEzGfO', '6WMo39FU3nrpSz3qMgRKug', '6UOwphPPKJ4W0G6SGtl2CB', '1tpXaFf2F55E7kVJON4j4G', '7EQ0qTo7fWT7DPxmxtSYEc', '3NPpFNZtSTHheNBaWC82rB', '3aVd0CcXBa0rOlxMpUS6on', '48vfT7i3EmuNEDAVRhzxpp', '1gl0S9pS0Zw0qfa14rDD3D', '3Sz7ZnJQBIHsXLUSo0OQtM', '3uwAm6vQy7kWPS2bciKWx9', '4j56EQDQu5XnL7R3E9iFJT', '4f9iBmdUOhQWeP7dcAn1pf', '1Ffb6ejR6Fe5IamqA5oRUF', '4U33udokhmKATsu8UoqUEN'

In [33]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
messages

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

# Tool Call

In [34]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "filter_artists":
    raise ValueError("should be tool message calling 'filter_artists'")

In [35]:
tool_response = tool_node.invoke({"messages": messages})
for tool_message in  tool_response["messages"]:
    tool_message.pretty_print()

================================= Tool Message =================================
Name: filter_artists

{'6dkpYaIjM9FjyCD9YGqgED', '1Ffb6ejR6Fe5IamqA5oRUF', '6XyY86QOPPrYVGvF9ch6wz', '4zYQWYmtimAEmI6WWEzGfO', '10exVja0key0uqUkk6LJRT', '3Sz7ZnJQBIHsXLUSo0OQtM', '1gl0S9pS0Zw0qfa14rDD3D', '5BtHciL0e0zOP7prIHn3pP', '6HySSUk2KthFj3bd9R61ql', '2ijef6ni2amuunRoKTlgww', '2txHhyCwHjUEpJjWrEyqyX', '7LIy7KinYq7a83dUH6kvxT', '1NlHgCXes12iEkqkXdvk4n', '1tpXaFf2F55E7kVJON4j4G', '3uwAm6vQy7kWPS2bciKWx9', '1JnWgHswLNLBcTnAOV8lA9', '40Yq4vzPs9VNUrIBG5Jr2i', '3aVd0CcXBa0rOlxMpUS6on', '7z2avKuuiMAT4XZJFv8Rvh', '2xiIXseIJcq3nG7C8fHeBj', '3NPpFNZtSTHheNBaWC82rB', '4f9iBmdUOhQWeP7dcAn1pf', '2MqhkhX4npxDZ62ObR5ELO', '4j56EQDQu5XnL7R3E9iFJT', '4axpPzNoUqm6nQ9K1VrdCu', '48vfT7i3EmuNEDAVRhzxpp', '7EQ0qTo7fWT7DPxmxtSYEc', '7JjdhWrqIaPVmplo9J8fEe', '4X07BN50otJgbMVTo4Jl2e', '6WMo39FU3nrpSz3qMgRKug', '1Jl8u1U1GtBlwocZK5LWZv', '0yb46jwm7gqbZXVXZQ8Z1e', '6u5mhJXgAKPTj6YVlZSPY9', '0XBRd3N11rCbh0s8qHf41m', '4U33udokhmK

In [36]:
chat_prompt_template += tool_response["messages"]
messages = chat_prompt_template.format_messages()
messages

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

# Send Tool Result to LLM

In [37]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

2024-11-15 12:38:07,246 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================

The filtered list of new artists not present in the 'New Rock and Blues' playlist includes:

1. Alice Merton
2. Awolnation
3. Bastille
4. Bishop Briggs
5. BØRNS
6. Chvrches
7. Daya
8. Dreamers
9. Empires
10. Fitz and The Tantrums
11. Glass Animals
12. Grimes
13. Hozier
14. Imagine Dragons
15. Kaleo
16. Kongos
17. Lorde
18. Mondo Cozmo
19. Nothing But Thieves
20. Of Monsters and Men
21. ODESZA
22. Saint Motel
23. The Fray
24. The Glorious Sons
25. The Neighbourhood
26. The Revivalists
27. Two Door Cinema Club
28. Walk The Moon
29. Young The Giant
30. The Revivalists (Innovation)
31. Vance Joy
32. Wolfmother
33. X Ambassadors
34. Young the Giant

Next, I will ensure these artists achieved significant success post-2010 and finalize the list to at least 40 new artists. Then, I will proceed to find top 3+ tracks for each, making sure the playlist reaches a minimum of 100 tracks, and create the playlist. Let's 

In [38]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
messages

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

In [39]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "find_top_tracks":
    raise ValueError("should be tool message calling 'find_top_tracks'")

# Tool Call

In [40]:
tool_response = tool_node.invoke({"messages": messages})
for tool_message in  tool_response["messages"]:
    tool_message.pretty_print()

2024-11-15 12:38:07,388 - ERROR - HTTP Error for GET to https://api.spotify.com/v1/artists/2ijef6ni2amuunRoKTlgww/top-tracks with Params: {'country': 'US'} returned 404 due to Resource not found
2024-11-15 12:38:07,389 - ERROR - HTTP Error for GET to https://api.spotify.com/v1/artists/6dkpYaIjM9FjyCD9YGqgED/top-tracks with Params: {'country': 'US'} returned 404 due to Resource not found


http status: 404, code:-1 - https://api.spotify.com/v1/artists/2ijef6ni2amuunRoKTlgww/top-tracks?country=US:
 Resource not found, reason: None
http status: 404, code:-1 - https://api.spotify.com/v1/artists/6dkpYaIjM9FjyCD9YGqgED/top-tracks?country=US:
 Resource not found, reason: None
================================= Tool Message =================================
Name: find_top_tracks

[]
================================= Tool Message =================================
Name: find_top_tracks

["0M3adYbGtyRHACP86dey1H", "0WSa1sucoNRcEeULlZVQXj", "7CAbF0By0Fpnbiu6Xn5ZF7", "7mczLeiFeYKRb7vtiPjuMb", "0hpWmAB3L0OJ3VBeMkOQUu", "6o39Ln9118FKTMbM4BvcEy", "0gchQwxmBWj5no8NJ8b2yH", "0Brlm0oPqzabpgIG1D1eU5", "25wdC7CJmCJPgnKw9rYquJ", "7g4hIeYVk3h5qQXb9RLeXP"]
================================= Tool Message =================================
Name: find_top_tracks

["2PnlsTsOTLE5jnBnNe2K0A", "60a0Rd6pjrkxjPbaKzXjfq", "2nLtzopw4rPReszdYBJU6h", "3fgehc497TFqKH1zBL2YNK", "4Yf5bqU3NK4kNOypcrLYwU", "3K4HG9

In [41]:
chat_prompt_template += tool_response["messages"]
messages = chat_prompt_template.format_messages()
messages

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

# Send Tool Result to LLM

In [42]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

2024-11-15 12:38:24,962 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  create_spotify_playlist (call_pIUX33ic0WKcZFst1k0oZXiq)
 Call ID: call_pIUX33ic0WKcZFst1k0oZXiq
  Args:
    name: Modern Rock and Blues Vibes
    description: A curated playlist featuring tracks from artists with similar styles and vibes as 'New Rock and Blues', focusing on post-2010 success.


In [43]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
messages

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

# Tool Call

In [44]:
tool_response = tool_node.invoke({"messages": messages})
for tool_message in  tool_response["messages"]:
    tool_message.pretty_print()

================================= Tool Message =================================
Name: create_spotify_playlist

{"id": "34Y803AoVL4ceg6jKNN2cz", "name": "Modern Rock and Blues Vibes"}


In [45]:
chat_prompt_template += tool_response["messages"]
messages = chat_prompt_template.format_messages()
messages

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

# Send Tool Result to LLM

In [46]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

2024-11-15 12:39:59,099 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  add_tracks_to_playlist (call_A7vf0b7b9Tg02jlArQOyteT7)
 Call ID: call_A7vf0b7b9Tg02jlArQOyteT7
  Args:
    playlist_id: 34Y803AoVL4ceg6jKNN2cz
    tracks: {'tracks': [{'id': '2PnlsTsOTLE5jnBnNe2K0A', 'uri': 'spotify:track:2PnlsTsOTLE5jnBnNe2K0A', 'name': 'Mad Hatter', 'artists': ['AWOLNATION'], 'album': 'Megalithic Symphony', 'duration_ms': 191706, 'explicit': False, 'popularity': 45}, {'id': '2nLtzopw4rPReszdYBJU6h', 'uri': 'spotify:track:2nLtzopw4rPReszdYBJU6h', 'name': 'Best Day Of My Life', 'artists': ['Imagine Dragons'], 'album': 'Smoke + Mirrors', 'duration_ms': 202940, 'explicit': False, 'popularity': 72}, {'id': '18lR4BzEs7e3qzc0KVkTpU', 'uri': 'spotify:track:18lR4BzEs7e3qzc0KVkTpU', 'name': 'Waves', 'artists': ['Dean Lewis'], 'album': 'A Place We Knew', 'duration_ms': 289066, 'explicit': False, 'popularity': 65}, {'id': '2vB9ME0oZwfTGAUMC71CaT', 'uri': 'spotify:track:2vB9ME0oZwfTGAUM